In [4]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import typing
from datetime import datetime

device = "cpu"

In [5]:
df = pd.read_csv("data.csv")
df.columns = ["Date", "Course"]
df = df.set_index('Date')
df.index = pd.to_datetime(df.index)
if not df.index.is_monotonic_increasing:
    df = df.sort_index()
df.dropna(axis=0, how="any", inplace=True)
print(df)

             Course
Date               
2015-01-01  56.2376
2015-01-13  62.7363
2015-01-14  64.8425
2015-01-15  66.0983
2015-01-16  64.8337
...             ...
2024-02-06  91.2434
2024-02-07  90.6842
2024-02-08  91.1514
2024-02-09  91.2561
2024-02-10  90.8901

[2237 rows x 1 columns]
